<a href="https://www.kaggle.com/code/abrarfairujraiyan/overcoming-output-token-limitation-for-docs-genera?scriptVersionId=199375159" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -q -U google-generativeai

Configure the library and start a chat with the model

In [16]:
import os
import google.generativeai as genai
from google.generativeai import caching
import datetime
import time

genai.configure(api_key='YOUR_API_KEY')

file = open('/kaggle/input/sample-long-code/trainer.py', 'r')
code = file.read()

model = genai.GenerativeModel('gemini-1.5-flash')
chat = model.start_chat(history=[])

combined_doc = ''

response = chat.send_message(f"""Code:{code}

You need to generate documentation for this code. Since this file is large, you will first generate the table of content. Later, you will generate documentation for the first few contents/titles from the table. Then, generate next few and next few and this will continue until whole table of content is covered.
Documentation requirement: Explain the parameters and algorithms for the functions

Start by generating the table of content only""")
combined_doc = response.text
print(response.text)


## Table of Contents

1. **BaseTrainer Class**
    - `__init__(self, cfg=DEFAULT_CFG, overrides=None, _callbacks=None)`
    - `add_callback(self, event: str, callback)`
    - `set_callback(self, event: str, callback)`
    - `run_callbacks(self, event: str)`
    - `train(self)`
    - `_setup_scheduler(self)`
    - `_setup_ddp(self, world_size)`
    - `_setup_train(self, world_size)`
    - `_do_train(self, world_size=1)`
    - `_get_memory(self)`
    - `_clear_memory(self)`
    - `read_results_csv(self)`
    - `save_model(self)`
    - `get_dataset(self)`
    - `setup_model(self)`
    - `optimizer_step(self)`
    - `preprocess_batch(self, batch)`
    - `validate(self)`
    - `get_model(self, cfg=None, weights=None, verbose=True)`
    - `get_validator(self)`
    - `get_dataloader(self, dataset_path, batch_size=16, rank=0, mode="train")`
    - `build_dataset(self, img_path, mode="train", batch=None)`
    - `label_loss_items(self, loss_items=None, prefix="train")`
    - `set_model_attributes

This table of content is not usable in code. So, turn it into a json array

In [19]:
import json

json_resp = chat.send_message("Now give me the table of content as a flat json array. ")

json_string = json_resp.text.replace('```json', '').replace('```', '').strip()
data = json.loads(json_string)
print(data)

['BaseTrainer Class', '__init__(self, cfg=DEFAULT_CFG, overrides=None, _callbacks=None)', 'add_callback(self, event: str, callback)', 'set_callback(self, event: str, callback)', 'run_callbacks(self, event: str)', 'train(self)', '_setup_scheduler(self)', '_setup_ddp(self, world_size)', '_setup_train(self, world_size)', '_do_train(self, world_size=1)', '_get_memory(self)', '_clear_memory(self)', 'read_results_csv(self)', 'save_model(self)', 'get_dataset(self)', 'setup_model(self)', 'optimizer_step(self)', 'preprocess_batch(self, batch)', 'validate(self)', 'get_model(self, cfg=None, weights=None, verbose=True)', 'get_validator(self)', 'get_dataloader(self, dataset_path, batch_size=16, rank=0, mode="train")', 'build_dataset(self, img_path, mode="train", batch=None)', 'label_loss_items(self, loss_items=None, prefix="train")', 'set_model_attributes(self)', 'build_targets(self, preds, targets)', 'progress_string(self)', 'plot_training_samples(self, batch, ni)', 'plot_training_labels(self)', '

Divide the list into multiple groups. The initial structure is not important.

In [20]:
divided_table = [data[i:i + 10] for i in range(0, len(data), 10)]

print(divided_table)

[['BaseTrainer Class', '__init__(self, cfg=DEFAULT_CFG, overrides=None, _callbacks=None)', 'add_callback(self, event: str, callback)', 'set_callback(self, event: str, callback)', 'run_callbacks(self, event: str)', 'train(self)', '_setup_scheduler(self)', '_setup_ddp(self, world_size)', '_setup_train(self, world_size)', '_do_train(self, world_size=1)'], ['_get_memory(self)', '_clear_memory(self)', 'read_results_csv(self)', 'save_model(self)', 'get_dataset(self)', 'setup_model(self)', 'optimizer_step(self)', 'preprocess_batch(self, batch)', 'validate(self)', 'get_model(self, cfg=None, weights=None, verbose=True)'], ['get_validator(self)', 'get_dataloader(self, dataset_path, batch_size=16, rank=0, mode="train")', 'build_dataset(self, img_path, mode="train", batch=None)', 'label_loss_items(self, loss_items=None, prefix="train")', 'set_model_attributes(self)', 'build_targets(self, preds, targets)', 'progress_string(self)', 'plot_training_samples(self, batch, ni)', 'plot_training_labels(self

Generate documentation part by part and merge them all together.

In [21]:
partial_resp = chat.send_message('Generate short documentation for the first 10 items in the table of content. Don\'t repeat the title')
# print(partial_resp.text)
combined_doc += partial_resp.text
divided_table.pop(0)

for i in divided_table:
    partial_resp = chat.send_message('Generate short documentation for the next ' + str(len(i)) + ' items in the table of content. Don\'t repeat the title')
    combined_doc += partial_resp.text
    # print(partial_resp.text)

print(combined_doc)

## Table of Contents

1. **BaseTrainer Class**
    - `__init__(self, cfg=DEFAULT_CFG, overrides=None, _callbacks=None)`
    - `add_callback(self, event: str, callback)`
    - `set_callback(self, event: str, callback)`
    - `run_callbacks(self, event: str)`
    - `train(self)`
    - `_setup_scheduler(self)`
    - `_setup_ddp(self, world_size)`
    - `_setup_train(self, world_size)`
    - `_do_train(self, world_size=1)`
    - `_get_memory(self)`
    - `_clear_memory(self)`
    - `read_results_csv(self)`
    - `save_model(self)`
    - `get_dataset(self)`
    - `setup_model(self)`
    - `optimizer_step(self)`
    - `preprocess_batch(self, batch)`
    - `validate(self)`
    - `get_model(self, cfg=None, weights=None, verbose=True)`
    - `get_validator(self)`
    - `get_dataloader(self, dataset_path, batch_size=16, rank=0, mode="train")`
    - `build_dataset(self, img_path, mode="train", batch=None)`
    - `label_loss_items(self, loss_items=None, prefix="train")`
    - `set_model_attributes